## 前向传播 单神经元 DL3
### 描述
实现一个单神经元的前向传播函数，使用sigmoid激活函数进行二分类预测。这是深度学习中最基本的神经网络单元。
$$mse=∑(predictions_i- lables_i)^2$$
 
### 输入描述：
函数接收4个参数：
1. features：二维列表，每行是一个样本的特征向量
2. labels：一维列表，包含对应的二分类标签（0或1）
3. weights：一维列表，权重向量
4. bias：浮点数，偏置值
### 输出描述：
返回一个元组，包含两个元素：
1. 预测概率列表：每个样本通过sigmoid函数后的预测概率（保留4位小数）
2. MSE值：预测概率与真实标签之间的均方误差（保留4位小数）

In [ ]:
import numpy as np
def single_neuron_model(features, labels, weights, bias):
    x = np.dot(features,weights)+bias
    probabilities = 1/(1+np.exp(-x))
    mse = np.mean((probabilities - np.array(labels))**2)
    return [round(p,4) for p in probabilities],round(mse,4)


if __name__ == "__main__":
    features = np.array([[1, 2], [2, 3]])
    labels = np.array([0,1])
    weights = np.array([0.5,0.5])
    bias = 0.0
    print(single_neuron_model(features, labels, weights, bias))


## 具有反向传播的单神经元 DL11 
### 描述
实现一个单神经元的训练函数，使用sigmoid激活函数和均方误差(MSE)损失，通过反向传播算法更新权重和偏置。这是深度学习中最基本的神经网络单元实现。
 

### 输入描述：
函数接收6个参数：
1. features：二维列表，每行是一个样本的特征向量
2. labels：一维列表，包含对应的二分类标签（0或1）
3. initial_weights：一维列表，初始权重
4. initial_bias：浮点数，初始偏置值
5. learning_rate：浮点数，学习率
6. epochs：整数，训练轮数
### 输出描述：
返回一个元组，包含三个元素：
1. 更新后的权重列表（保留4位小数）
2. 更新后的偏置值（保留4位小数）
3. 每个epoch的MSE值列表（保留4位小数）

In [ ]:
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivate(x):
    return x * (1-x)

def forward(x,y,w,b):
    out = np.dot(x,w)+b
    pred = sigmoid(out)
    mse = np.mean((pred-labels)**2)
    return pred,mse

def backward(x,y,dy,w,b,lr):
    dz = dy * sigmoid_derivate(y)
    dw = np.dot(x.T,dz)*2/ len(x) # 2 mse 没有 /2，所以这里加2 平均梯度
    db = np.sum(dz)*2/ len(x)

    w -= dw * lr
    b -= db * lr
    return w,b

def train_neuron(features, labels, initial_weights, initial_bias, learning_rate, epochs):
    w = initial_weights
    b = initial_bias
    mses = []

    for epoch in range(epochs):
        pred,loss = forward(features,labels,w,b)
        mses.append(round(loss,4))
        dy =  pred - labels
        w,b = backward(features,pred,dy,w,b,learning_rate)
    return [round(ww,4) for ww in w],round(b,4),mses



if __name__ == "__main__":
    features = np.array([[1, 2], [2, 3]])
    labels = np.array([0,1])
    initial_weights = np.array([0.5,0.5])
    initial_bias = 0.0
    learning_rate = 0.1
    epochs = 2
    print(train_neuron(features, labels, initial_weights, initial_bias, learning_rate, epochs))



https://blog.csdn.net/weixin_54856108/article/details/140878689
反向传播算法的主要步骤如下：

计算输出层误差：根据误差函数计算预测输出与真实值之间的误差。
计算隐藏层误差：利用链式法则将误差反向传递到各隐藏层。
更新权重和偏置：根据计算出的梯度调整每个神经元的权重和偏置。


- dw 和 db 的计算方式都来源于梯度下降的 均值梯度 计算，而 db 可以使用 np.sum(dz)，但通常使用 np.mean(dz) 使其梯度更稳定。如果不除以 len(self.X)，梯度更新可能会 随着样本数量增加而变大，导致训练不稳定。np.sum 梯度下降更新的步长会更大（相当于 lr * len(self.X),lr需要缩小 以补偿db过大
- 乘 2 是因为 MSE 的导数带有 2(y - t)
- 如果使用 交叉熵损失（如二元交叉熵 BCE），它的导数不会有 2，所以 dw 也不需要乘 2。

- 如果 MSE 公式里有 1/2，梯度计算时不用乘 2。
- 如果 MSE 公式里没有 1/2，梯度计算时需要乘 2。
- 两种方式最终效果一样，只是第一种（带 1/2）让梯度公式更简洁，不需要额外乘 2，所以常见于数学推导。
- 代码实现时，可能为了简化代码而不加 1/2，这样计算梯度时就必须额外乘 2。

In [1]:
import numpy as np

# Sigmoid 激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Sigmoid 导数（用于反向传播）
def sigmoid_derivative(x):
    return x * (1 - x)

class Neuron:
    def __init__(self, input_size):
        """
        input_size: 输入特征的维度
        """
        # 初始化权重和偏置
        self.w = np.random.randn(input_size)  # 权重: 向量 (input_size,)
        self.b = np.random.randn()  # 偏置: 标量

    def forward(self, X):
        """前向传播: 计算神经元输出"""
        self.X = np.array(X)  # 输入样本 (样本数, 特征数)
        self.z = np.dot(self.X, self.w) + self.b  # 计算线性组合 (样本数,)
        self.y = sigmoid(self.z)  # 计算激活值 (样本数,)
        return self.y

    def backward(self, dy, lr=0.1):
        """反向传播: 计算梯度并更新参数"""
        dz = dy * sigmoid_derivative(self.y)  # 计算对 z 的梯度 (样本数,)

        # 计算梯度
        dw = np.dot(self.X.T, dz) / len(self.X)  # (特征数,) - 均值梯度
        db = np.mean(dz)  # (标量) - 偏置梯度

        # 更新权重和偏置
        self.w -= lr * dw
        self.b -= lr * db

    def train(self, X, target, epochs=1000, lr=0.1):
        """训练神经元 (支持多个样本)"""
        X = np.array(X)  # (样本数, 特征数)
        target = np.array(target)  # (样本数,)

        for epoch in range(epochs):
            y_pred = self.forward(X)  # 前向传播
            loss = np.mean(0.5 * (y_pred - target) ** 2)  # 均方误差 (MSE)
            dy = y_pred - target  # 计算损失对输出的梯度
            self.backward(dy, lr)  # 反向传播

            if epoch % 100 == 0:
                print(f"Epoch {epoch}: Loss = {loss:.4f}, w = {self.w}, b = {self.b:.4f}")

# 3 维输入，4 个样本
X = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [2, 3, 4]
]  # (4, 3) 样本矩阵

target = [1, 0, 1, 0]  # (4,) 目标输出

# 训练神经元
input_size = 3
neuron = Neuron(input_size)
neuron.train(X, target, epochs=1000, lr=0.1)

# 训练后测试
print("Final outputs:", neuron.forward(X))


Epoch 0: Loss = 0.1970, w = [ 0.55513493 -0.74698749 -0.13639967], b = 3.6322
Epoch 100: Loss = 0.1705, w = [ 0.85969309 -0.73424896 -0.41548077], b = 3.3403
Epoch 200: Loss = 0.1470, w = [ 1.17626542 -0.69422327 -0.6520017 ], b = 3.0638
Epoch 300: Loss = 0.1326, w = [ 1.42651293 -0.65878009 -0.83136286], b = 2.8490
Epoch 400: Loss = 0.1255, w = [ 1.60201554 -0.63337484 -0.95605497], b = 2.6989
Epoch 500: Loss = 0.1222, w = [ 1.7211743  -0.61608743 -1.04063892], b = 2.5970
Epoch 600: Loss = 0.1207, w = [ 1.80276829 -0.60428467 -1.09862738], b = 2.5272
Epoch 700: Loss = 0.1200, w = [ 1.85977831 -0.59607013 -1.13920832], b = 2.4784
Epoch 800: Loss = 0.1196, w = [ 1.90042816 -0.59023394 -1.16818578], b = 2.4436
Epoch 900: Loss = 0.1194, w = [ 1.9299146  -0.58601313 -1.18923062], b = 2.4184
Final outputs: [0.39453113 0.51582874 0.63528961 0.43427633]
